In [2]:
%load_ext autoreload
%autoreload 2

In [6]:
from openqaoa.qaoa_parameters import create_qaoa_variational_params, QAOACircuitParams, PauliOp, Hamiltonian
from openqaoa.utilities import X_mixer_hamiltonian, qaoa_probabilities
from openqaoa.devices import DevicePyquil,create_device
from openqaoa.backends.qpus.qaoa_pyquil_qpu import QAOAPyQuilQPUBackend
from openqaoa.backends.qaoa_backend import get_qaoa_backend
from openqaoa.backends.simulators.qaoa_vectorized import QAOAvectorizedBackendSimulator

from pyquil import Program, gates, quilbase
from pyquil.gates import RX, RY, RZ

import numpy as np
import matplotlib.pyplot as plt

In [8]:
cost_hamil = Hamiltonian([PauliOp('Z',(0,)), PauliOp('Z',(1,)), PauliOp('ZZ',(1,0))], [1,1,2], 1)
mixer_hamil = X_mixer_hamiltonian(n_qubits=2)
circuit_params = QAOACircuitParams(cost_hamil, mixer_hamil, p=1)
variate_params = create_qaoa_variational_params(circuit_params,'standard','ramp')

device_pyquil = create_device('qcs',"Aspen-11", as_qvm=True, execution_timeout = 10, compiler_timeout=10)
backend_obj_pyquil = get_qaoa_backend(circuit_params, device_pyquil, n_shots=1000)

vect_sim = create_device('local','vectorized')
backend_obj_vectorized = get_qaoa_backend(circuit_params,vect_sim)

In [9]:
mixer_hamil.qureg

[0, 1]

In [10]:
cost_hamil.qureg

[0, 1]

In [11]:
cost_hamil.qureg

[0, 1]

In [12]:
print(backend_obj_pyquil.qaoa_circuit(variate_params))

DECLARE ro BIT[2]
DECLARE pauli1qcost00 REAL[1]
DECLARE pauli1qcost01 REAL[1]
DECLARE pauli2qcost00 REAL[1]
DECLARE pauli1qmixer00 REAL[1]
DECLARE pauli1qmixer01 REAL[1]
RZ(pi) 0
RX(pi/2) 0
RZ(pi/2) 0
RX(-pi/2) 0
RZ(1.0*pauli2qcost00[0] + 1.0*pauli1qcost00[0]) 0
RZ(pi) 1
RX(pi/2) 1
RZ(pi/2) 1
RX(-pi/2) 1
RZ(1.0*pauli2qcost00[0] + 1.0*pauli1qcost01[0]) 1
CPHASE(-2.0*pauli2qcost00[0]) 0 1
RZ(pi/2) 1
RX(pi/2) 1
RZ(1.0*pauli1qmixer01[0]) 1
RX(-pi/2) 1
RZ(-pi/2) 1
MEASURE 1 ro[1]
RZ(pi/2) 0
RX(pi/2) 0
RZ(1.0*pauli1qmixer00[0]) 0
RX(-pi/2) 0
RZ(-pi/2) 0
MEASURE 0 ro[0]
HALT



In [13]:
variate_params.update_from_raw([1,1])
backend_obj_pyquil.expectation(variate_params)

3.944